<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/AdiabaticFlash_Broyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [2]:
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)
import tools.che as che
R=8.314 # J/(mol K)
eps=1e-12

In [3]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
Ftot=10 # Total Feed moles
Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
FT = 450 # Feed temperature
flashP= 101325 # Flash drum pressure

Vy = Fz # Guess vapor/liquid composition equal to feed
Lx = Fz # Comp - constrains mole fractions to behave like mole fractions!
flashT = jnp.array([360.])  # Guess and bounds for flash temperature
Vtot = jnp.array([Ftot/2])  # Guess half of feed in vapor
Ltot = jnp.array([Ftot/2])

x_guess = jnp.concatenate([Vy, Lx, flashT, Vtot, Ltot])

In [5]:
def model(x):
  Vy = x[0:3]
  Lx = x[3:6]
  flashT = x[6]
  Vtot = x[7]
  Ltot = x[8]

  sum_frac = jnp.array([jnp.sum(Vy)-1., jnp.sum(Lx)-1.])

  V = Vy * Vtot
  L = Lx * Ltot
  F = Fz * Ftot
  mass_balance = F-V-L

  FH = p.Hl(nL=F, T=FT)
  VH = p.Hv(nV=V, T=flashT)
  LH = p.Hl(nL=L, T=flashT)
  energy_balance = jnp.array([FH - VH - LH])

  fugL = Lx  * p.NRTL_gamma(Lx,flashT)* p.Pvap(flashT)
  fugV = Vy*flashP
  VLE = fugL - fugV
  return jnp.concatenate([sum_frac, mass_balance, energy_balance, VLE])

In [6]:
model(x_guess)

DeviceArray([     0.        ,      0.        ,      0.        ,
                  0.        ,      0.        , -48834.08861275,
              14479.44772707,  11725.37795764,   6218.03624225],            dtype=float64)

In [7]:
def broyden3(func, x, J=None, max_iter=100, tol=1e-6, verbose=0, xmax=jnp.inf, xmin=-jnp.inf):
  Jf = jax.jacobian(func) if J is None else J
  J = Jf(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    print(f"\nIter: {i}")
    dx = - Jinv @ f

    alpha = jnp.min(jnp.concatenate([jnp.abs((xmax-x)/dx), jnp.abs((xmin-x)/dx), jnp.array([1.])]))

    while alpha > 0.01:
      dx_try = alpha*dx
      xp = x + dx_try
      fp = func(xp)
      dnorm = jnp.linalg.norm(fp)-jnp.linalg.norm(f)
      if verbose>1:
        print(f"Alpha {alpha}   dnorm {dnorm}  dx_try {dx_try}   f {f}    fp {fp}")
      if dnorm > 0:
        alpha *= 0.5
      else:
        break
    if alpha <= 0.01:
      if verbose>1:
        print("reevaluate J")
      Jinv = jnp.linalg.inv(Jf(x))
      continue

    dx=dx_try
    f= fp
    x= xp
    if verbose>0:
      print(i, x, f)
      print()
    if jnp.linalg.norm(fp) < tol:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

In [8]:
broyden3(model, x_guess, verbose=2, max_iter=100,
         xmin=jnp.array([0., 0., 0., 0., 0., 0., 300., 0., 0.]),
         xmax=jnp.array([1., 1., 1., 1., 1., 1., FT, Ftot, Ftot]))


Iter: 0
Alpha 1.0   dnorm -50722.25969292458  dx_try [ 1.36747385e-02  5.53176044e-03 -1.92064989e-02 -1.36747385e-02
 -5.53176044e-03  1.92064989e-02 -6.38009102e+00 -1.06008684e+00
  1.06008684e+00]   f [     0.              0.              0.              0.
      0.         -48834.08861275  14479.44772707  11725.37795764
   6218.03624225]    fp [-1.11022302e-16 -2.22044605e-16  2.89928208e-02  1.17282929e-02
 -4.07211137e-02  3.87246605e+02  1.42338377e+03  1.08312861e+03
  5.60423287e+02]
0 [3.47008072e-01 3.38865094e-01 3.14126834e-01 3.19658595e-01
 3.27801573e-01 3.52539832e-01 3.53619909e+02 3.93991316e+00
 6.06008684e+00] [-1.11022302e-16 -2.22044605e-16  2.89928208e-02  1.17282929e-02
 -4.07211137e-02  3.87246605e+02  1.42338377e+03  1.08312861e+03
  5.60423287e+02]


Iter: 1
Alpha 1.0   dnorm -1697.6875310282678  dx_try [ 0.00493122  0.0011131  -0.00604433  0.00144166  0.00146488 -0.00290653
 -0.6767386   0.02989179 -0.02989179]   f [-1.11022302e-16 -2.22044605e-16  2.8992

(DeviceArray([3.51709766e-01, 3.39621207e-01, 3.08669027e-01,
              3.21226469e-01, 3.29190721e-01, 3.49582810e-01,
              3.52854975e+02, 3.97163878e+00, 6.02836122e+00],            dtype=float64),
 DeviceArray([ 0.00000000e+00,  0.00000000e+00, -1.56523683e-11,
               1.14148690e-11,  4.23794333e-12, -3.50875780e-07,
               2.81965185e-07, -4.02746082e-07,  1.27995008e-07],            dtype=float64))